# Tigergraph Queries

## Overview

Experimenting with Tigergraph queries

In [1]:
import os
import pyTigerGraph as tg
from dotenv import load_dotenv
import json
import pandas as pd
import requests

load_dotenv()

print("Import successful.")

Import successful.


In [8]:
tg_graph = 'marites'
tg_host = os.environ.get("TG_HOST").strip('/')
tg_secret = os.environ.get("TG_SECRET")
tg_password = os.environ.get("TG_PASSWORD")

In [3]:
token = tg.TigerGraphConnection(host=tg_host, graphname=tg_graph).getToken(tg_secret)[0]
conn = tg.TigerGraphConnection(host=tg_host, graphname=tg_graph, password=tg_password, apiToken=token)

In [9]:
print(tg_host)

https://marites.i.tgcloud.io


In [14]:
url = f'{tg_host}:9000/query/{tg_graph}/get_following?p=elonmusk'
print(url)
res = requests.get(url, headers={ 'Authorization': f'Bearer {token}' })
res.json()

https://marites.i.tgcloud.io:9000/query/marites/get_following?p=elonmusk


{'version': {'edition': 'enterprise', 'api': 'v2', 'schema': 0},
 'error': False,
 'message': '',
 'results': [{'tgt': [{'v_id': 'WorldAndScience',
     'v_type': 'user',
     'attributes': {'name': 'World and Science',
      'username': 'WorldAndScience'}},
    {'v_id': '4thFromOurStar',
     'v_type': 'user',
     'attributes': {'name': 'Mars', 'username': '4thFromOurStar'}},
    {'v_id': 'CoinDesk',
     'v_type': 'user',
     'attributes': {'name': 'CoinDesk', 'username': 'CoinDesk'}},
    {'v_id': 'HGJart',
     'v_type': 'user',
     'attributes': {'name': '黄光剑', 'username': 'HGJart'}},
    {'v_id': 'paraga',
     'v_type': 'user',
     'attributes': {'name': 'Parag Agrawal', 'username': 'paraga'}},
    {'v_id': 'nichegamer',
     'v_type': 'user',
     'attributes': {'name': 'Niche Gamer', 'username': 'nichegamer'}},
    {'v_id': 'SpaceX',
     'v_type': 'user',
     'attributes': {'name': 'SpaceX', 'username': 'SpaceX'}},
    {'v_id': 'universal_sci',
     'v_type': 'user',
   

In [15]:
url = f'{tg_host}:9000/query/{tg_graph}/get_user_topics?u=elonmusk'
print(url)
res = requests.get(url, headers={ 'Authorization': f'Bearer {token}' })
res.json()

https://marites.i.tgcloud.io:9000/query/marites/get_user_topics?u=elonmusk


{'version': {'edition': 'enterprise', 'api': 'v2', 'schema': 0},
 'error': False,
 'message': '',
 'results': [{'all_topics': [{'v_id': 'Lux',
     'v_type': 'topic',
     'attributes': {'text': 'Lux', 'topic_type': 'ORGANIZATION'}},
    {'v_id': 'Banks',
     'v_type': 'topic',
     'attributes': {'text': 'Banks', 'topic_type': 'ORGANIZATION'}},
    {'v_id': '@MahowaldARK',
     'v_type': 'topic',
     'attributes': {'text': '@MahowaldARK', 'topic_type': 'ORGANIZATION'}},
    {'v_id': 'businesses',
     'v_type': 'topic',
     'attributes': {'text': 'businesses', 'topic_type': 'ORGANIZATION'}},
    {'v_id': 'company',
     'v_type': 'topic',
     'attributes': {'text': 'company', 'topic_type': 'ORGANIZATION'}},
    {'v_id': 'Bitcoin',
     'v_type': 'topic',
     'attributes': {'text': 'Bitcoin', 'topic_type': 'ORGANIZATION'}},
    {'v_id': 'Gmail',
     'v_type': 'topic',
     'attributes': {'text': 'Gmail', 'topic_type': 'ORGANIZATION'}},
    {'v_id': 'Transformer',
     'v_type': '

In [4]:
q = conn.gsql("select count(*) from user")
q

'[{\n"count": 115,\n"v_type": "user"\n}]'

In [5]:
conn.gsql('select * from user where primary_id=="elonmusk"')

'[{\n"v_id": "elonmusk",\n"attributes": {\n"name": "Elon Musk",\n"username": "elonmusk"\n},\n"v_type": "user"\n}]'

## get_following query

```
CREATE QUERY get_user_topics(VERTEX<user> u) FOR GRAPH marites {
  start = {u};
  all_topics = SELECT t 
               FROM user:s-(created_post:cp)-post:p-(topic_sentiment:ts)-topic:t 
               WHERE ts.sentiment == "NEUTRAL" OR ts.sentiment == "POSITIVE";
  PRINT all_topics;
}
```

In [25]:
print(conn.gsql('''
use graph marites

drop query get_following

create query get_following(vertex<user> p) for graph marites {
    start = {p};
    tgt = select t from start:s-(following:e)-user:t;
    print tgt;
}

install query get_following
'''))

Using graph 'marites'
Successfully dropped queries on the graph 'marites': [hello].
Successfully created queries: [hello].
Start installing queries, about 1 minute ...
hello query: curl -X GET 'https://127.0.0.1:9000/query/marites/hello?p=VALUE'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation finished.


In [7]:
q = conn.gsql('run query get_following({})'.format('"joerogan"'))
print(q)

{
"error": false,
"message": "",
"version": {
"schema": 0,
"edition": "enterprise",
"api": "v2"
},
"results": [{"tgt": []}]
}


In [4]:
print(conn.gsql('''
use graph marites

drop query get_user_topics

create query get_user_topics(vertex<user> u) for graph marites {
    start = {u};
    all_topics = select t
                 from user:s-(created_post>:cp)-post:p-(topic_sentiment>:ts)-topic:t
                 where (ts.sentiment == "POSITIVE") and (t.topic_type == "ATTRIBUTE" or t.topic_type == "ORGANIZATION");
    print all_topics;
}

install query get_user_topics
'''))

Using graph 'marites'
Semantic Check Fails: These queries could not be found anywhere: [get_user_topics].
Successfully created queries: [get_user_topics].
Start installing queries, about 1 minute ...
get_user_topics query: curl -X GET 'https://127.0.0.1:9000/query/marites/get_user_topics?u=VALUE'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation finished.


In [16]:
topic_res_str = conn.gsql('run query get_user_topics("{}")'.format('elonmusk'))

In [17]:
topic_res = json.loads(topic_res_str)

In [18]:
topics = list(map(lambda x: x['attributes'], topic_res['results'][0]['all_topics']))

In [19]:
topic_df = pd.DataFrame(topics)

In [20]:
print(topic_df.head())
print(topic_df.shape)

           text    topic_type
0           Lux  ORGANIZATION
1         Banks  ORGANIZATION
2  @MahowaldARK  ORGANIZATION
3    businesses  ORGANIZATION
4       company  ORGANIZATION
(100, 2)


In [25]:
# Clean the data

pronouns = [
    'i',
    'we',
    'you',
    'he',
    'she',
    'it',
    'they',
    'me',
    'us',
    'him',
    'her',
    'them',
    'ours',
    'yours',
    'his',
    'its',
    'hers',
    'ours',
    'theirs',
    'myself',
    'yourself',
    'himself',
    'herself',
    'ourselves',
    'itself',
    'themselves',
    'each other',
    'one another',
    'that',
    'which',
    'who',
    'whom',
    'whose',
    'where',
    'when',
    'what',
    'whatever',
    'anything',
    'anybody',
    'anyone',
    'something',
    'somebody',
    'nothing',
    'nobody',
    'none',
    'no one'
]

# Remove all symbols
topic_df.text = topic_df.text.str.replace('[^0-9a-zA-Z]', '')

# Convert everything to lowercase
topic_df.text = topic_df.text.str.lower()

# Remove pronouns
topic_df = topic_df[~topic_df.text.isin(pronouns)]

# Remove numbers
topic_df = topic_df[~topic_df.text.str.contains(r'\d')]

# Remove blank strings
topic_df = topic_df[topic_df.text.str.strip().str.len() != 0]


list(topic_df.text)

/var/folders/4f/zf_kr4_16jvb5njtdm05rm440000gn/T/ipykernel_18011/3400499045.py:53: FutureWarning: The default value of regex will change from True to False in a future version.
  topic_df.text = topic_df.text.str.replace('[^0-9a-zA-Z]', '')


['lux',
 'banks',
 'mahowaldark',
 'businesses',
 'company',
 'bitcoin',
 'gmail',
 'transformer',
 'nasawebbtelescope',
 'force',
 'hifglobal',
 'society',
 'almere',
 'zune',
 'dogecoin',
 'sls',
 'team',
 'tokens',
 'ford',
 'microsoft',
 'waterfront',
 'design',
 'bluetooth',
 'insaitinstitute',
 'yoodli',
 'mercury',
 'teslagigafactory',
 'vendors',
 'pricedrop',
 'amazonworkers',
 'pretend',
 'openai',
 'miri',
 'tesla',
 'view',
 'volkswagen',
 'nat',
 'columbia',
 'colour',
 'jll',
 'org',
 'booster',
 'senpower',
 'bitpay',
 'cnn',
 'communitycrypto',
 'science',
 'ratio',
 'alessionaval',
 'design',
 'skorusark',
 'near',
 'elonmusk',
 'doge',
 'game',
 'tagboard',
 'duncantrussell',
 'team',
 'views',
 'marshelicopter',
 'teslas',
 'webb',
 'opdivo',
 'aceattorney',
 'ltum',
 'quality',
 'windpower',
 'biotech',
 'netflix',
 'ux',
 'nasamoon',
 'bcmspacehealth',
 'userexperience',
 'content',
 'heart',
 'governorva',
 'ark',
 'parkis',
 'emudeck',
 'foundation',
 'amazon',
 